In [159]:
import numpy.linalg as lin
import scipy.integrate as intgr
import numpy as np
import pandas as pd
import random
import scipy.stats as sts
from scipy.stats import uniform


import requests
import statistics
import matplotlib.pyplot as plt
import scipy.special as spc
import scipy.optimize as opt
import seaborn as sns

sns.set()

In [160]:
# import os
# os.getcwd()

In [161]:
#load data and reshape it
macroseries = np.loadtxt('data/NewMacroSeries.txt', delimiter=',')
df = pd.DataFrame(macroseries)
df.columns = ['c', 'k', 'w', 'r', 'y']
df.head()

,c,k,w,r,y
0,1.128323e+07,8.040697e+06,1.120211e+07,1.008852,1.931398e+07
1,1.215464e+07,8.030754e+06,1.206726e+07,1.088112,2.080561e+07
2,1.097303e+07,8.650974e+06,1.089414e+07,0.911904,1.878300e+07
3,9.711635e+06,7.809971e+06,9.641815e+06,0.893986,1.662382e+07
4,9.245673e+06,6.912184e+06,9.179203e+06,0.961637,1.582621e+07


In [162]:
# #Initial vals
# alpha_0=0.3
# rho_0=0.4
# mu_0=0.7
# sigma_0=0.6
# beta=0.99 #given
# np.random.seed(100)

# #Parameters
# alpha=alpha_0
# #beta=beta_0
# rho=rho_0
# mu=mu_0
# sigma=sigma_0


# Simulate the data

In [163]:
# #assumptions (Given). Fixed across simulations
# k_1 = df['k'].mean()


# #simulation parameters
# err_sigma = 0.1
# alpha_s = 0.5
# beta = 0.99
# mu_s = 10
# z_0 = mu_s
# rho_s = 0.7
# err_sigma = 0.1
# k_mean = df['k'].mean()

In [164]:
#Simulate data
def get_sim_vals(alpha_s, mu_s, err_sigma, rho_s):
    np.random.seed(100)
    T = 100 #100 
    S = 1000 #1000
    #draw u
    u_st = uniform.rvs(size=(T, S))
    #calc eps
    np.random.seed(100)
    eps_st = sts.norm.ppf(u_st, 0, err_sigma)

    #calc z_st
    z_st = np.zeros((T, S))
    z_st[0,:] = z_0 + eps_st[0,:]
    for i in range(1,T):
        for j in range(0, S):
            z_st[i,j] = rho_s*z_st[i-1,j] + (1-rho_s)*mu_s + eps_st[i,j] #5

    #calc k_st
    k_st = np.zeros((T, S))
    k_st[0,:] = df.k.mean()
    for i in range(0,T-1):
        for j in range(0, S):
            k_st[i+1,j] = alpha_s*beta*np.exp(z_st[i,j])*k_st[i,j]**alpha_s #7

    # k_test  = np.zeros((T, S))
    # k_test[0,:] = df.k.mean()
    # for i in range(1,T):
    #     for j in range(0, S):
    #         k[i,j] = alpha_s*beta*np.exp(z[i,j])*k[i,j]**alpha_s #7




    #calc w_st, r_st, c_st, y_st
    w_st = np.zeros((T, S))
    r_st = np.zeros((T, S))
    c_st = np.zeros((T, S))
    y_st = np.zeros((T, S))

    for i in range(0,T):
        for j in range(0, S):
            w_st[i,j] = (1-alpha_s)*np.exp(z_st[i,j])*k_st[i,j]**alpha_s #3
            r_st[i,j] = alpha_s * np.exp(z_st[i,j])*k_st[i,j]**(alpha_s - 1) #4
            y_st[i,j] = np.exp(z_st[i,j])*k_st[i,j]**alpha_s
    for i in range(0, T-1):
        for j in range(0,S):
            c_st[i,j] = r_st[i,j]*k_st[i,j] + w_st[i,j] - k_st[i+1,j] #2
            
    return c_st, k_st, w_st, r_st, y_st

# #winsoriz last value of c
# c[T-1,:] = r


#         if i != T-1: #Upper bound
#             c[i,j] = r[i,j]*k[i,j] + w[i,j] - k[i+1,j] #2 confused how this works with k?
#         if i == T-1:
#             c[i,j] = c[i-1,j]





#note all values are now stored in arrays k, w, r, y, c


# part a

In [165]:
#create c_t-1 and store in same df as c_t, for calculation of moments
c_data = pd.DataFrame(df.c.values)
c_df = pd.concat([c_data.shift(-1), c_data], axis=1, )
c_df.columns = ['c_t-1', 'c_t']
c_df.head()


,c_t-1,c_t
0,1.215464e+07,1.128323e+07
1,1.097303e+07,1.215464e+07
2,9.711635e+06,1.097303e+07
3,9.245673e+06,9.711635e+06
4,8.065031e+06,9.245673e+06


In [166]:
def moments_calc(c, k, w, r, y):
    #calc c_lag
    c_lag = pd.DataFrame(c).shift(1)
    c_lag_array = np.asarray(c)
    #for 1D data (data moments recognized)
    if c.ndim == 1:
        m1 = c.mean()
        m2 = k.mean()
        m3 = (c/y).mean()
        m4 = y.var()
        m5 = sts.pearsonr(c, c_lag_array)[0] # c has dim 6, clag has 100
        m6 = sts.pearsonr(c, k)[0]
    elif c.ndim ==2:
        m1 = statistics.mean(c.mean(axis=0))
        m2 = statistics.mean(k.mean(axis=0))
        m3 = statistics.mean((c/y).mean(axis=0))
        m4 = statistics.mean(y.var(axis=0))

        c_mean = c.mean(axis=1)
        k_mean = k.mean(axis=1)
        c_lag_mean = c_lag_array.mean(axis=1)
        m5 = sts.pearsonr(c_mean, c_lag_mean)[0]
        m6 = sts.pearsonr(c_mean, k_mean)[0]
#         m5 = np.dot(c.mean(axis=0), c_lag.T)
#         m6 = np.dot(c.mean(axis=0), k.mean(axis=0).T)
    
    return m1, m2, m3, m4, m5, m6

In [169]:
def err_vec(sim_vals, simple):
    c_st, k_st, w_st, r_st, y_st = sim_vals
    moms_data =  moments_calc(macroseries[0,:],macroseries[0,:],macroseries[0,:],macroseries[0,:],macroseries[0,:])
    moms_model = moments_calc(c_st, k_st, w_st, r_st, y_st)
    

    if simple:
#         #case if m1 - m4 (scalars)
#         for i in range (0,6):
#             if moms_model[i].ndim ==0:


        err_vec = np.subtract(moms_model, moms_data)
#             elif moms_model[i].ndim == 2:
                
    else: #check this case
        #case if m1 - m4 (scalars)
        err_vec = np.divide(np.subtract(moms_model,moms_data), moms_data)
        
        #case if m5 - m6 (non-scalars)
    return err_vec

In [170]:
def criterion(params, *args):
    alpha, mu, sigma, rho = params
    sim_vals, W_hat = args
    sim_vals = get_sim_vals(alpha, mu, sigma, rho)
    err = err_vec(sim_vals, simple=True)#returns an array
    crit_val = err.T @ W_hat @ err 
    return crit_val

In [171]:
#Estimate by SMM
#Initial parameters
alpha_init = 0.5
beta = 0.99
mu_init =  10
sigma_init = 0.1
rho_init = 0.7

params_init = np.array([alpha_init, mu_init, sigma_init, rho_init])
W_hat = np.eye(6) #6 for number of moments
# data = c, k, w, r, y
sim_init = get_sim_vals(alpha_init, mu_init, sigma_init, rho_init)
smm_args = (sim_init, W_hat)
results = opt.minimize(criterion, params_init, args=(smm_args), method='L-BFGS-B', bounds=((0.01-(1e-10), 0.99+(1e-10)), (5-(1e-10), 14+(1e-10)), (0.01-(1e-10), 1.1+(1e-10)), (-0.99-(1e-10), 0.99+(1e-10))))
alpha_smm, mu_smm, sigma_smm, rho_smm = results.x
print('alpha_smm=', alpha_smm, 'mu_smm=', mu_smm, 'sigma_smm=', sigma_smm, 'rho_smm=', rho_smm)

alpha_smm= 0.5062479198266697 mu_smm= 5.9487798994111305 sigma_smm= 0.010002908339340331 rho_smm= -0.7050907921830913


In [173]:
results

      fun: 183423106142700.7
 hess_inv: <4x4 LbfgsInvHessProduct with dtype=float64>
      jac: array([ 1.44186862e+22,  6.51571736e+16, -5.60120719e+17,  2.22458001e+15])
  message: b'ABNORMAL_TERMINATION_IN_LNSRCH'
     nfev: 890
      nit: 21
   status: 2
  success: False
        x: array([ 0.50624792,  5.9487799 ,  0.01000291, -0.70509079])

In [175]:
#Vector of moment differences at optimum
#compute simulated data moments with 2d data
c_opt, k_opt, w_opt, r_opt, y_opt = get_sim_vals(alpha_smm, mu_smm, sigma_smm, rho_smm)
sim_m1, sim_m2, sim_m3, sim_m4, sim_m5, sim_m6 = moments_calc(c_opt, k_opt, w_opt, r_opt, y_opt)
print("C moment mean = ", sim_m1)
print("K moment mean = ", sim_m2)
print("C moment data mean = ", sim_m3)
print("Y moment mean = ", sim_m4)
#preview of correlations due to length
print("\nC and C lagged value moment correlation\n", sim_m5)
print("\nC and K  moment correlation\n", sim_m6)

#compute data moments with 1 dimensinosla data
data_m1, data_m2, data_m3, data_m4, data_m5, data_m6 = moments_calc(macroseries[:,0], macroseries[:,1], macroseries[:,2], macroseries[:,3], macroseries[:,4])
print("C data mean = ", data_m1)
print("K data mean = ", data_m2)
print("C share data mean = ", data_m3)
print("Y data mean = ", data_m4)
print("C and C lagged value data correlation", data_m5)
print("C and K  data correlation", data_m6)

#Vector of moment differences
mom_diff = np.array([sim_m1 - data_m1, sim_m2 - data_m2, sim_m3 - data_m3, sim_m4 - data_m4, sim_m5 - data_m5, sim_m6 - data_m6])
print("Vector of moment differences at optimum: ", mom_diff)

C data mean =  357989.46114641335
K data mean =  426130.8476743191
C share data mean =  0.49382641377788017
Y data mean =  38735153259660.445

C and C lagged value data correlation
 1.0

C and K  data correlation
 0.19716164818699167
C data mean =  9281790.485669706
K data mean =  6643985.138299068
C share data mean =  0.5842000000000002
Y data mean =  28377825058899.727
C and C lagged value data correlation 0.9999999999999999
C and K  data correlation 0.9408030537975819
Vector of moment differences at optimum:  [-8.92380102e+06 -6.21785429e+06 -9.03735862e-02  1.03573282e+13
  1.11022302e-16 -7.43641406e-01]


In [176]:
print("Criterion function value: ", results.fun)
#183423106142700.7 for alpha_s, mu_s, err_sigma, rho_s as initials
#128984461473121.52 for 0.5, 8, 0.5, 0 but leads to errors below (check before changing values)

Criterion function value:  183423106142700.7


In [177]:
def Jac_err2(alpha, mu, sigma, rho, simple=True):
    '''
    This function computes the Jacobian matrix of partial derivatives of the R x 1 moment
    error vector e(x|theta) with respect to the K parameters theta_i in the K x 1 parameter vector
    theta. The resulting matrix is R x K Jacobian.
    '''
    Jac_err = np.zeros((6, 4))
    h_alpha = 1e-4 * alpha
    h_mu = 1e-4 * mu
    h_sig = 1e-4 * sigma
    h_rho = 1e-4 * rho

    Jac_err[:, 0] = \
        ((err_vec(get_sim_vals(alpha + h_alpha, mu, sigma, rho), simple) -
          err_vec(get_sim_vals(alpha - h_alpha, mu, sigma, rho), simple)) / (2 * h_alpha)).flatten()
    Jac_err[:, 1] = \
        ((err_vec(get_sim_vals(alpha, mu + h_mu, sigma, rho), simple) -
          err_vec(get_sim_vals(alpha, mu - h_mu, sigma, rho), simple)) / (2 * h_mu)).flatten()  
    Jac_err[:, 2] = \
    ((err_vec(get_sim_vals(alpha, mu, sigma + h_sig, rho), simple) -
      err_vec(get_sim_vals(alpha, mu, sigma - h_sig, rho), simple)) / (2 * h_sig)).flatten()
    Jac_err[:, 3] = \
    ((err_vec(get_sim_vals(alpha, mu, sigma, rho + h_rho), simple) -
      err_vec(get_sim_vals(alpha, mu, sigma, rho - h_rho), simple)) / (2 * h_rho)).flatten()


    return Jac_err

In [150]:
# S = unif_vals_2.shape[1]
S = 1000
d_err2 = Jac_err2(alpha_smm, mu_smm, sigma_smm, rho_smm, False)
print("d_err2", d_err2)
print("Weight matrix", W_hat)
SigHat2 = (1 / S) * lin.inv(d_err2.T @ W_hat @ d_err2)
print("SigHat2", SigHat2)
print('Std. err. alpha_hat=', np.sqrt(SigHat2[0, 0]))
print('Std. err. mu_hat=', np.sqrt(SigHat2[1, 1]))
print('Std. err. sigma_hat=', np.sqrt(SigHat2[2, 2]))
print('Std. err. rho_hat=', np.sqrt(SigHat2[3, 3]))

d_err2 [[ 5.30200462e-01  8.93802046e-03  5.24723202e-04 -1.46734569e-03]
 [ 6.75616217e-01  8.98050315e-03  5.27217314e-04 -1.47432003e-03]
 [-9.80100000e-01  0.00000000e+00  0.00000000e+00 -0.00000000e+00]
 [ 3.13969136e+01 -5.46379938e-03  5.05260020e-02 -1.87741202e-04]
 [ 1.09652107e-12  9.33151876e-14  5.54950114e-11  7.87290826e-13]
 [-3.53112210e+00 -1.38229837e-03 -3.18526088e-03  1.00258921e-02]]
Weight matrix [[1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 1.]]
SigHat2 [[ 1.03012372e-03 -5.71539226e-03 -6.40148594e-01  1.58644435e-01]
 [-5.71539226e-03  6.82334874e+00  4.29407945e+00  1.96161517e+00]
 [-6.40148594e-01  4.29407945e+00  3.98280724e+02 -9.81146839e+01]
 [ 1.58644435e-01  1.96161517e+00 -9.81146839e+01  3.52217563e+01]]
Std. err. alpha_hat= 0.032095540523940286
Std. err. mu_hat= 2.6121540412164843
Std. err. sigma_hat= 19.95697182678933
Std. err. rho_hat= 5.934792014873299


# part b

In [151]:
def get_Err_mat2(sim_vals, alpha, mu, sigma, rho, simple=True):
    c_st, k_st, w_st, r_st, y_st = sim_vals
    R = 6
    S = 1000
    Err_mat = np.zeros((R, S))
    m1_data, m2_data, m3_data, m4_data, m5_data, m6_data = moments_calc(macroseries[:,0], macroseries[:,1], macroseries[:,2], macroseries[:,3], macroseries[:,4])
    m1_model, m2_model, m3_model, m4_model, m5_model, m6_model = moments_calc(c_st, k_st, w_st, r_st, y_st)
    if simple:
        Err_mat[0, :] = m1_model - m1_data
        Err_mat[1, :] = m2_model - m2_data
        Err_mat[2, :] = m3_model - m3_data
        Err_mat[3, :] = m4_model - m4_data
        Err_mat[4, :] = m5_model - m5_data
        Err_mat[5, :] = m6_model - m6_data


    else:
        Err_mat[0, :] = (m1_model - m1_data) / m1_data
        Err_mat[1, :] = (m2_model - m2_data) / m2_data
        Err_mat[2, :] = (m3_model - m3_data) / m3_data
        Err_mat[3, :] = (m4_model - m4_data) / m4_data
        Err_mat[4, :] = (m5_model - m5_data) / m5_data
        Err_mat[5, :] = (m6_model - m6_data) / m6_data
    
    return Err_mat

In [152]:
sim_vals_2 = get_sim_vals(alpha_smm, mu_smm, sigma_smm, rho_smm)
Err_mat2 = get_Err_mat2(sim_vals_2, alpha_smm, mu_smm, sigma_smm, rho_smm, True)
VCV2 = (1 / 1000) * (Err_mat2 @ Err_mat2.T)
print(VCV2)
W_hat2_1 = lin.inv(VCV2)
print(W_hat2_1)

[[ 7.96342247e+13  5.54868945e+13  8.06475901e+05 -9.24267360e+19
  -9.90740936e-10  6.63610794e+06]
 [ 5.54868945e+13  3.86617120e+13  5.61929791e+05 -6.44003576e+19
  -6.90320500e-10  4.62385390e+06]
 [ 8.06475901e+05  5.61929791e+05  8.16738509e-03 -9.36028893e+11
  -1.00334836e-17  6.72055407e-02]
 [-9.24267360e+19 -6.44003576e+19 -9.36028893e+11  1.07274247e+26
   1.14989442e-03 -7.70213810e+12]
 [-9.90740936e-10 -6.90320500e-10 -1.00334836e-17  1.14989442e-03
   1.23259516e-32 -8.25607811e-17]
 [ 6.63610794e+06  4.62385390e+06  6.72055407e-02 -7.70213810e+12
  -8.25607811e-17  5.53002540e-01]]
[[ 5.45142482e+00  3.29270823e+00  1.78754226e+08 -2.05005061e-06
   6.89821200e+23 -4.02387221e+07]
 [ 3.40046051e+00 -6.67743478e-02 -3.47248999e+07  3.57915323e-07
   4.23917986e+22 -2.47137188e+07]
 [ 1.51520907e+08 -4.47756127e+07  2.69458614e+16 -1.15549935e+02
   2.55396771e+31 -2.51497986e+15]
 [-1.84417920e-06  4.03428183e-07 -1.14521226e+02  3.10961096e-14
  -2.12525445e+17  1.378

In [153]:
#Estimate by SMM
#Initial parameters


params_init_2 = np.array([alpha_smm, mu_smm, sigma_smm, rho_smm])
# data = c, k, w, r, y
sim_init_2 = get_sim_vals(alpha_smm, mu_smm, sigma_smm, rho_smm)
# sim = get_sim_vals(#PARAMS)
smm_args_2 = (sim_init_2, W_hat2_1)
results_2 = opt.minimize(criterion, params_init_2, args=(smm_args_2), method='L-BFGS-B', bounds=((0.01-(1e-10), 0.99+(1e-10)), (5-(1e-10), 14+(1e-10)), (0.01-(1e-10), 1.1+(1e-10)), (-0.99-(1e-10), 0.99+(1e-10))))
alpha_smm_2, mu_smm_2, sigma_smm_2, rho_smm_2 = results_2.x


In [154]:
print('alpha_smm=', alpha_smm_2, 'mu_smm=', mu_smm_2, 'sigma_smm=', sigma_smm_2, 'rho_smm=', rho_smm_2)

alpha_smm= 0.5062479198266697 mu_smm= 5.9487798994111305 sigma_smm= 0.010002908339340331 rho_smm= -0.7050907921830913


In [178]:
print("value of criterion function", results_2.fun)

value of criterion function 6390971898184170.0


In [180]:
#Vector of moment differences at optimum
#compute simulated data moments with 2d data
c_opt, k_opt, w_opt, r_opt, y_opt = get_sim_vals(alpha_smm_2, mu_smm_2, sigma_smm_2, rho_smm_2)
sim_m1, sim_m2, sim_m3, sim_m4, sim_m5, sim_m6 = moments_calc(c_opt, k_opt, w_opt, r_opt, y_opt)
print("C moments mean = ", sim_m1)
print("K moments mean = ", sim_m2)
print("C share moments mean = ", sim_m3)
print("Y moments mean = ", sim_m4)
#preview of correlations due to length
print("\nC and C lagged value moments correlation\n", sim_m5)
print("\nC and K  moments correlation\n", sim_m6)

#compute data moments with 1 dimensinosla data
data_m1, data_m2, data_m3, data_m4, data_m5, data_m6 = moments_calc(macroseries[:,0], macroseries[:,1], macroseries[:,2], macroseries[:,3], macroseries[:,4])
print("C data mean = ", data_m1)
print("K data mean = ", data_m2)
print("C share data mean = ", data_m3)
print("Y data mean = ", data_m4)
print("C and C lagged value data correlation", data_m5)
print("C and K  data correlation", data_m6)

#Vector of moment differences
mom_diff = np.array([sim_m1 - data_m1, sim_m2 - data_m2, sim_m3 - data_m3, sim_m4 - data_m4, sim_m5 - data_m5, sim_m6 - data_m6])
print("Vector of moment differences at optimum: ", mom_diff)

C moments mean =  357989.46114641335
K moments mean =  426130.8476743191
C share moments mean =  0.49382641377788017
Y moments mean =  38735153259660.445

C and C lagged value moments correlation
 1.0

C and K  moments correlation
 0.19716164818699167
C data mean =  9281790.485669706
K data mean =  6643985.138299068
C share data mean =  0.5842000000000002
Y data mean =  28377825058899.727
C and C lagged value data correlation 0.9999999999999999
C and K  data correlation 0.9408030537975819
Vector of moment differences at optimum:  [-8.92380102e+06 -6.21785429e+06 -9.03735862e-02  1.03573282e+13
  1.11022302e-16 -7.43641406e-01]


In [156]:
# S = unif_vals_2.shape[1]
S = 1000
d_err2 = Jac_err2(alpha_smm_2, mu_smm_2, sigma_smm_2, rho_smm_2, False)
print("d_err2", d_err2)
print("Weight matrix", W_hat2_1)
SigHat2 = (1 / S) * lin.inv(d_err2.T @ W_hat2_1 @ d_err2)
print("SigHat2", SigHat2)
print('Std. err. alpha_hat=', np.sqrt(SigHat2[0, 0]))
print('Std. err. mu_hat=', np.sqrt(SigHat2[1, 1]))
print('Std. err. sigma_hat=', np.sqrt(SigHat2[2, 2]))
print('Std. err. rho_hat=', np.sqrt(SigHat2[3, 3]))

d_err2 [[ 5.30200462e-01  8.93802046e-03  5.24723202e-04 -1.46734569e-03]
 [ 6.75616217e-01  8.98050315e-03  5.27217314e-04 -1.47432003e-03]
 [-9.80100000e-01  0.00000000e+00  0.00000000e+00 -0.00000000e+00]
 [ 3.13969136e+01 -5.46379938e-03  5.05260020e-02 -1.87741202e-04]
 [ 1.09652107e-12  9.33151876e-14  5.54950114e-11  7.87290826e-13]
 [-3.53112210e+00 -1.38229837e-03 -3.18526088e-03  1.00258921e-02]]
Weight matrix [[ 5.45142482e+00  3.29270823e+00  1.78754226e+08 -2.05005061e-06
   6.89821200e+23 -4.02387221e+07]
 [ 3.40046051e+00 -6.67743478e-02 -3.47248999e+07  3.57915323e-07
   4.23917986e+22 -2.47137188e+07]
 [ 1.51520907e+08 -4.47756127e+07  2.69458614e+16 -1.15549935e+02
   2.55396771e+31 -2.51497986e+15]
 [-1.84417920e-06  4.03428183e-07 -1.14521226e+02  3.10961096e-14
  -2.12525445e+17  1.37880013e+00]
 [ 6.62166503e+23  2.98061051e+22  2.69919573e+31 -2.24853955e+17
   6.98362971e+46 -4.18109024e+30]
 [-3.90914295e+07 -2.34443196e+07 -2.69467457e+15  2.51426177e+00
  -4.